In [9]:
# generate links / json states programatically by cell type or anything, probably hack-y but it works
from caveclient import CAVEclient
import datetime
import pandas as pd
import numpy as np
import json

client = CAVEclient()
datastack_name = 'fanc_production_mar2021'
client = CAVEclient(datastack_name)
client.materialize.version = 604
timestamp = datetime.datetime.fromtimestamp(1685002201.254304)

In [10]:
ngl_base = 'https://neuromancer-seung-import.appspot.com/' # this will point to the public bucket


In [18]:
def make_json2(seg_ids=None, hidden_ids=None,name="published FANC neurons"):
    state = {
    "layers": [
      {
        "source": "precomputed://gs://lee-lab_female-adult-nerve-cord/alignmentV4/em/rechunked",
        "type": "image",
        "blend": "default",
        "shaderControls": {},
        "name": "FANC EM"
      },
      {
        "type": "segmentation",
        "mesh": "precomputed://gs://lee-lab_female-adult-nerve-cord/meshes/FANC/FANC_neurons/meshes",
        "selectedAlpha": 0.4,
        "colorSeed": 3788587020,
        "segments": seg_ids,
        "hiddenSegments": hidden_ids,
        "skeletonRendering": {
          "mode2d": "lines_and_points",
          "mode3d": "lines"
        },
        "name": name #"published FANC neurons"
      },
      {
        "type": "segmentation",
        "mesh": "precomputed://gs://lee-lab_female-adult-nerve-cord/alignmentV4/volume_meshes/meshes",
        "objectAlpha": 0.1,
        "hideSegmentZero": False,
        "ignoreSegmentInteractions": True,
        "segmentColors": {
          "1": "#bfbfbf",
          "2": "#d343d6"
        },
        "segments": [
          "1",
          "2"
        ],
        "skeletonRendering": {
          "mode2d": "lines_and_points",
          "mode3d": "lines"
        },
        "name": "region outlines"
      }
    ],
    "navigation": {
      "pose": {
        "position": {
          "voxelSize": [
            4.300000190734863,
            4.300000190734863,
            45
          ],
          "voxelCoordinates": [38734, 148426, 2200
          ]
        }
      },
      "zoomFactor": 12
    },
    "showAxisLines": False,
    "showDefaultAnnotations": False,
    "perspectiveZoom": 6062.41070084089,
    "showSlices": False,
    "gpuMemoryLimit": 4000000000,
    "systemMemoryLimit": 4000000000,
    "concurrentDownloads": 64,
    "jsonStateServer": "https://global.daf-apis.com/nglstate/api/v1/post",
    "selectedLayer": {
      "layer": "published FANC neurons",
      "visible": True
    },
    "layout": "3d"
  }

    return state

In [19]:
# import dataframe
leg_df = pd.read_pickle('pre_to_mn_df_matched_typed_with_nt_v604_20230525.pkl')
wing_df = pd.read_pickle('preMN_to_MN_wing_v604.pkl')

In [20]:
print(leg_df.shape)
leg_df = leg_df[leg_df.index.get_level_values('cell_type') != 'Gial_error']
print(leg_df.shape)

(1460, 69)
(1460, 69)


# Preferred pool for different classes neurons

In [21]:
# Preferred pool for intersegmental neurons
# generate a link based on a preferred_pool
# local and intersegmental preMNs
All = slice(None)
leg_df = leg_df.loc[leg_df.index]

cell_class = leg_df.index.get_level_values('cell_class').unique()
idx = pd.IndexSlice
for cc in cell_class:

    class_df = leg_df.loc[idx[cc]]
    pool_keys = class_df.index.get_level_values('preferred_pool').unique()

    for pl in pool_keys:
        seg_ids = class_df.loc[pl,:].index.get_level_values('segID')
        formatted_ids = [str(x) for x in seg_ids]
        
        state = make_json2(name = pl,
                    seg_ids = formatted_ids)
        new_id = client.state.upload_state_json(state)
        client.state.build_neuroglancer_url(new_id, ngl_base)

        # save json 
        with open('jsons/leg_jsons/{}_{}.json'.format(cc,pl), "w") as outfile:
            json.dump(state, outfile)

# Make Jsons for hemilineages

In [22]:
leg_df = pd.read_pickle('pre_to_mn_df_matched_typed_with_nt_v604_20230525.pkl')
cell_type_dict = {
    'RVD':'8A',
    '8A_':'8A',
    '16B_':'16B',
    '16B_small':'16B',
    'Rcore_':'8A',
    '24B_':'15B',
    '8A-':'8A',
    '22A_':'22A',
    '8A? 16B?':'16A',
    '21A_':'19A',
    '9Ac_':'9A',
    '13B_':'13B',
    '1B_':'1B',
    '22A?':'22A',
    'R0':'15B',
    'R1':'8A',
    'R2':'8A',
    'R3':'8A',
    'R4':'8A',
    'R5':'8A',
    'R6':'8A',
    '4B_':'4B',
    '4B-':'4B',
    '4B+':'4B',
    '14A_':'14A',
    'RCore':'8A',
    '8A- Rind':'8A',    
    '8A+':'8A',
}
cell_types = leg_df.index.get_level_values('cell_type').unique().to_numpy()
leg_idx = leg_df.index.to_frame().set_index('cell_type',drop=False)

for ct in cell_type_dict.keys():
    leg_idx.loc[ct,'cell_type'] = cell_type_dict[ct]

cell_types_ = leg_idx['cell_type'].unique()

leg_df.index = pd.MultiIndex.from_frame(leg_idx)
class_df = leg_df.loc['local']
cell_types = class_df.index.get_level_values('cell_type').unique().to_numpy()
print(cell_types)
# generate a link based on a hemilineage
# local and intersegmental preMNs

class_df = leg_df.loc['local']
for ct in cell_types:
    if type(ct) is not str and np.isnan(ct):
        continue
    seg_ids = class_df.loc[idx[:,:,:,ct],:].index.get_level_values('segID')
    formatted_ids = [str(x) for x in seg_ids]
    
    state = make_json2(name = ct,
                seg_ids = formatted_ids)
    new_id = client.state.upload_state_json(state)
    client.state.build_neuroglancer_url(new_id, ngl_base)

    # save json 
    with open('jsons/leg_jsons/{}.json'.format(ct), "w") as outfile:
        json.dump(state, outfile)

# seg_ids = class_df.loc[idx[:,:,:,ct],:].index.get_level_values('segID')
# formatted_ids = [str(x) for x in seg_ids]

# state = make_json(name = ct,
#             seg_ids = formatted_ids)
# new_id = client.state.upload_state_json(state)
# client.state.build_neuroglancer_url(new_id, ngl_base)

# # save json 
# with open('leg_jsons/{}.json'.format(ct), "w") as outfile:
#     json.dump(state, outfile)

['21A' '8A' '3A' '16B' '20A' '13A' '4B' '15B' '14A' '23B' nan '22A' '16A'
 '1A' '6A' '6B' '19A' '13B' '9A' '19B' '1B']


# Make motor module states

In [23]:
class_df

side                                                                                                                            L  \
nerve                                                                                                                      Dorsal   
segment                                                                                                                    thorax   
function                                                                                                                    swing   
muscle                                                                        tergopleural_promotor_pleural_promotor_miller_28_30   
rank                                                                                                                            0   
segID                                                                                                          648518346487756866   
preferred_pool        NT   classification_system cell_type segID                                                                    
thorax_swing          Glu  B                     21A       648518346491498682                                                 11    
                           R                     8A        648518346517460010                                                  0    
                      Ach  C                     3A        648518346467065838                                                  0    
                      Glu  Q                     16B       648518346487752258                                                142    
                                                           648518346488061982                                                 27    
...                                                                                                                           ...   
tarsus_depressor_noid GABA D                     13A       648518346499795084                                                  0    
                      Ach  J                     20A       648518346467213319                                                  0    
                      NaN  NaN                   NaN       648518346490452234                                                  0    
                      Glu  P                     14A       648518346493240384                                                  0    
                                                           648518346482229908                                                  0    

side                                                                                              \
nerve                                                                                              
segment                                                                                            
function                                                                                           
muscle                                                                                             
rank                                                                                               
segID                                                                         648518346491659326   
preferred_pool        NT   classification_system cell_type segID                                   
thorax_swing          Glu  B                     21A       648518346491498682                 19   
                           R                     8A        648518346517460010                  7   
                      Ach  C                     3A        648518346467065838                 25   
                      Glu  Q                     16B       648518346487752258                106   
                                                           648518346488061982                 40   
...                                                                                          ...   
tarsus_depressor_noid GABA D                     13A       6485183464997

In [24]:
import utils
mpool_dict = utils.get_motor_pool_tuple_dict()
pool_keys = [
    'thorax_swing',
    'thorax_stance',
    'trochanter_extension',
    'trochanter_flexion',
    'femur_reductor',
    'tibia_extensor',
    # 'main_tibia_flexor',      # main_tibia_flexor for both main and A groups, or main_tibia_flexor_muscle for the muscle
    'aux_main_tibia_flexor_wtarsus',
    'auxiliary_tibia_flexor_B_wtarsus',
    'auxiliary_tibia_flexor_E_wtarsus',
    'ltm',
    'tarsus_depressor_med_venU',
    ]
for p in pool_keys:
    sids = class_df.loc[:,mpool_dict[p]].columns.get_level_values('segID')
    state = make_json2(name = pl,
                    seg_ids = formatted_ids)
    
class_df = leg_df.loc['local']
for p in pool_keys:
    if type(ct) is not str and np.isnan(ct):
        continue
    seg_ids = class_df.loc[:,mpool_dict[p]].columns.get_level_values('segID')
    formatted_ids = [str(x) for x in seg_ids]
    
    state = make_json2(name = p,
                seg_ids = formatted_ids)
    new_id = client.state.upload_state_json(state)
    client.state.build_neuroglancer_url(new_id, ngl_base)

    # save json 
    with open('jsons/leg_jsons/mn_mod_{}.json'.format(p), "w") as outfile:
        json.dump(state, outfile)

In [803]:
# if you want a link
client.state.build_neuroglancer_url(new_id, ngl_base)

'https://neuromancer-seung-import.appspot.com/?json_url=https://global.daf-apis.com/nglstate/api/v1/5986275473489920'